## Feature engineering

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

from feature_engineering.indicators import (
    simple_moving_average,
    momentum,
    cci,
    williams_r,
    exponential_moving_average,
    bollinger_bands,
    macd,
    atr,
    obv
)
from feature_engineering.time_based_features import add_time_based_features

In [4]:
data = pd.read_csv('../data/binance/BTC/1h.csv')

In [5]:
data.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
0,2024-09-17 19:00:00,60540.17,60696.48,59618.26,59967.99,2112.05320,2024-09-17 19:59:59.999,1.270612e+08,322417,881.13029,5.299674e+07,0
1,2024-09-17 20:00:00,59967.98,60342.85,59715.42,60120.00,1665.06151,2024-09-17 20:59:59.999,9.996920e+07,208296,828.74091,4.975709e+07,0
2,2024-09-17 21:00:00,60119.99,60424.25,60105.01,60334.07,662.68788,2024-09-17 21:59:59.999,3.997326e+07,85064,356.50218,2.150287e+07,0
3,2024-09-17 22:00:00,60334.07,60343.99,60122.50,60240.01,424.37655,2024-09-17 22:59:59.999,2.556103e+07,94677,173.20763,1.043171e+07,0
4,2024-09-17 23:00:00,60240.01,60324.00,60005.31,60313.99,489.52738,2024-09-17 23:59:59.999,2.945738e+07,73556,267.66050,1.610952e+07,0


In [6]:
# Ensure data is sorted by date
data['Date'] = pd.to_datetime(data['Open Time'])
data.sort_values('Date', inplace=True)

In [7]:
# Apply Simple Moving Average (SMA)
data = simple_moving_average(data, 'Close', window=5)
data = simple_moving_average(data, 'Close', window=10)

# Apply Momentum (MTM)
data = momentum(data, 'Close', window=5)
data = momentum(data, 'Close', window=10)

# Apply Exponential Moving Average (EMA)
data = exponential_moving_average(data, 'Close', span=5)
data = exponential_moving_average(data, 'Close', span=10)

# Apply Bollinger Bands (BB)
data = bollinger_bands(data, 'Close', window=20)

# Apply Moving Average Convergence Divergence (MACD)
data = macd(data, 'Close')

# Apply Average True Range (ATR)
data = atr(data, window=14)

# Apply On-Balance Volume (OBV)
data = obv(data)

# Apply Commodity Channel Index (CCI)
data = cci(data, window=20)

# Apply Williams %R
data = williams_r(data, window=14)

# Apply time-based features
data = add_time_based_features(data, 'Date')

In [8]:
data.dropna(inplace=True)

## Create Lagged Features

Use past data points as features to predict the next data points

In [9]:
for lag in range(1, 6):
    data[f'Close_lag_{lag}'] = data['Close'].shift(lag)
                                                   
data.dropna(inplace=True)
data.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,...,CCI_20,Williams_%R_14,Day_of_Week,Hour_of_Day,Month_of_Year,Close_lag_1,Close_lag_2,Close_lag_3,Close_lag_4,Close_lag_5
24,2024-09-18 19:00:00,60629.79,60745.99,59987.25,60057.99,3156.74947,2024-09-18 19:59:59.999,1.905570e+08,753066,1532.69879,...,43.420706,-58.802594,2,19,9,60629.79,60013.01,59915.26,59429.18,59487.60
25,2024-09-18 20:00:00,60057.99,60320.00,59473.68,60230.01,2083.48272,2024-09-18 20:59:59.999,1.247339e+08,286619,995.87495,...,-12.175255,-50.778524,2,20,9,60057.99,60629.79,60013.01,59915.26,59429.18
26,2024-09-18 21:00:00,60230.01,60496.95,60168.04,60199.46,808.61440,2024-09-18 21:59:59.999,4.877547e+07,104108,372.10512,...,47.715405,-52.203564,2,21,9,60230.01,60057.99,60629.79,60013.01,59915.26
27,2024-09-18 22:00:00,60199.46,60700.00,60194.00,60684.78,732.26197,2024-09-18 22:59:59.999,4.429355e+07,105072,401.74056,...,92.793425,-29.565258,2,22,9,60199.46,60230.01,60057.99,60629.79,60013.01
28,2024-09-18 23:00:00,60684.78,61786.24,60680.00,61759.99,2346.43342,2024-09-18 23:59:59.999,1.437302e+08,252632,1465.86349,...,236.260632,-1.005193,2,23,9,60684.78,60199.46,60230.01,60057.99,60629.79


### Split the data into features and target

In [10]:
features = data[['Close_lag_1', 'Close_lag_2', 'Close_lag_3', 'Close_lag_4', 'Close_lag_5',
                 'SMA_5', 'SMA_10', 'MTM_5', 'MTM_10', 'EMA_5', 'EMA_10', 'MA', 'UB', 'LB', 'MACD',
                 'Signal_Line', 'ATR_14', 'OBV', 'CCI_20', 'Williams_%R_14', 
                 'Day_of_Week', 'Hour_of_Day', 'Month_of_Year']]
target = data['Close']

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

### Preprocessing

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4]
}

gbr = GradientBoostingRegressor()

grid_search = GridSearchCV(estimator=gbr,
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           scoring='neg_mean_squared_error'
                           )

grid_search.fit(X_train_scaled, y_train)

best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')

Best parameters found: {'learning_rate': 0.2, 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}


In [14]:
best_gbr = GradientBoostingRegressor(**best_params)
best_gbr.fit(X_train_scaled, y_train)

y_pred = best_gbr.predict(X_test_scaled)
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'R^2 Score: {r2_score(y_test, y_pred)}')

Mean Squared Error: 20440.70058473591
R^2 Score: 0.9948791163020775


In [15]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(best_gbr, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')

# Convert negative MSE to positive
cv_scores = -cv_scores

print(f'Cross-Validation MSE: {cv_scores}')
print(f'Mean Cross-Validation MSE: {cv_scores.mean()}')
print(f'Standard Deviation of Cross-Validation MSE: {cv_scores.std()}')

Cross-Validation MSE: [41983.22590207 26576.49434041 14778.19296578 22831.36532188
 34364.50435841]
Mean Cross-Validation MSE: 28106.756577712134
Standard Deviation of Cross-Validation MSE: 9375.903919845885


### Try Randomized Search for better performance

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# Define the parameter distribution for Random Search
param_dist = {
    'n_estimators': randint(100, 500),
    'learning_rate': uniform(0.01, 0.1),
    'max_depth': randint(3, 10),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10)
}

# Initialize the model
gbr = GradientBoostingRegressor()

# Initialize Random Search
random_search = RandomizedSearchCV(estimator=gbr, param_distributions=param_dist, n_iter=100, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)

# Perform Random Search
random_search.fit(X_train_scaled, y_train)

# Get the best parameters
best_params = random_search.best_params_
print(f'Best parameters found: {best_params}')

# Train the model with the best parameters
best_gbr = GradientBoostingRegressor(**best_params)
best_gbr.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = best_gbr.predict(X_test_scaled)
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred)}')
print(f'R^2 Score: {r2_score(y_test, y_pred)}')

# Perform cross-validation with the best model
cv_scores = cross_val_score(best_gbr, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
cv_scores = -cv_scores

print(f'Cross-Validation MSE: {cv_scores}')
print(f'Mean Cross-Validation MSE: {cv_scores.mean()}')
print(f'Standard Deviation of Cross-Validation MSE: {cv_scores.std()}')

Best parameters found: {'learning_rate': 0.08982951789667752, 'max_depth': 3, 'min_samples_leaf': 8, 'min_samples_split': 9, 'n_estimators': 499}
Mean Squared Error: 16653.56804130144
R^2 Score: 0.9958278834552947
Cross-Validation MSE: [38291.62241157 22161.40759631 13659.94983642 23732.57386811
 27806.11560194]
Mean Cross-Validation MSE: 25130.333862868894
Standard Deviation of Cross-Validation MSE: 8033.8943243080885


### Try TimeSeriesSplit Cross Validation

more approriate because data is time-series

In [17]:
from sklearn.model_selection import TimeSeriesSplit

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Perform cross-validation with TimeSeriesSplit
cv_scores = cross_val_score(best_gbr, X_train_scaled, y_train, cv=tscv, scoring='neg_mean_squared_error')
cv_scores = -cv_scores

print(f'TimeSeriesSplit Cross-Validation MSE: {cv_scores}')
print(f'Mean TimeSeriesSplit Cross-Validation MSE: {cv_scores.mean()}')
print(f'Standard Deviation of TimeSeriesSplit Cross-Validation MSE: {cv_scores.std()}')

TimeSeriesSplit Cross-Validation MSE: [72980.3578514  23096.20706707 17856.10947758 24515.62351105
 29408.89624149]
Mean TimeSeriesSplit Cross-Validation MSE: 33571.43882971748
Standard Deviation of TimeSeriesSplit Cross-Validation MSE: 20045.44590409118


### Experiment with Xgboost

### Makes it worse!!

In [18]:
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

# Initialize the XGBoost model
xgbr = xgb.XGBRegressor(objective='reg:squarederror')

# Perform cross-validation with TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
cv_scores = cross_val_score(xgbr, X_train_scaled, y_train, cv=tscv, scoring='neg_mean_squared_error')
cv_scores = -cv_scores

print(f'TimeSeriesSplit Cross-Validation MSE with XGBoost: {cv_scores}')
print(f'Mean TimeSeriesSplit Cross-Validation MSE with XGBoost: {cv_scores.mean()}')
print(f'Standard Deviation of TimeSeriesSplit Cross-Validation MSE with XGBoost: {cv_scores.std()}')

# Train the XGBoost model with the entire training set
xgbr.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = xgbr.predict(X_test_scaled)
print(f'Mean Squared Error with XGBoost: {mean_squared_error(y_test, y_pred)}')
print(f'R^2 Score with XGBoost: {r2_score(y_test, y_pred)}')

TimeSeriesSplit Cross-Validation MSE with XGBoost: [94425.54289248 40811.75333099 24151.84912017 31771.73160418
 48462.51841967]
Mean TimeSeriesSplit Cross-Validation MSE with XGBoost: 47924.67907349551
Standard Deviation of TimeSeriesSplit Cross-Validation MSE with XGBoost: 24654.729670505287
Mean Squared Error with XGBoost: 26678.551456901136
R^2 Score with XGBoost: 0.9933163856750657
